Alejandro Rodríguez Mesa, Visión por Computador


Tarea - Pintar círculos en las posiciones del píxel más claro y oscuro de la imagen

Paquetes necesarios

In [5]:

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt


Inicializaciones

In [6]:
#Dimensiones de la imagen a crear
ancho = 600
alto = 600


Funcion para encontrar el pixel más claro y el más oscuro

In [7]:
def find_brightest_and_darkest_pixel(image):
    """
    Encuentra el píxel más brillante de la imagen.
    :param image: Imagen de entrada en formato BGR
    :return: Coordenadas del píxel más claro y su valor de brillo promedio
    """
    # Convertir la imagen a escala de grises para facilitar el cálculo del brillo
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    
    # Encontrar el valor máximo de brillo y su posición
    darkest_value, brightest_value, coord_darkest, coord_brightest = cv.minMaxLoc(gray_image)
    
    return darkest_value, brightest_value, coord_darkest, coord_brightest

Pintar círculos en el pixel más claro y el más oscuro.

In [13]:
# Bucle principal del video en tiempo real
video = cv.VideoCapture(0)
while True:
    # Leer el frame de la cámara
    ret, image = video.read()
    height, weight, _ = image.shape
    font = cv.FONT_HERSHEY_SIMPLEX
    fontScale = 0.5
    color_dark = (255, 0, 0)
    color_bright = (0, 255, 0)
    thickness = 2

    # Asegurarse de que el frame se haya leído correctamente
    if not ret:
        print("Error al acceder a la cámara.")
        break
        # Configuración de texto

    # Encontrar los píxeles más claros y más oscuros en la imagen
    darkest_value, brightest_value, coord_darkest, coord_brightest = find_brightest_and_darkest_pixel(image)
    
    # Dibujar un círculo verde en el píxel más claro
    cv.circle(image, coord_brightest, 8, (0, 255, 0), -1)

    # Dibujar un círculo azul en el píxel más oscuro
    cv.circle(image, coord_darkest, 8, (255, 0, 0), -1)

    text_claro = f"Coord{coord_darkest} - Pixel mas claro: {brightest_value}"  
    text_oscuro = f"Coord{coord_brightest} - Pixel mas oscuro: {darkest_value}" 
    textsize = cv.getTextSize(text_oscuro, font, fontScale=fontScale, thickness=thickness)[0]

    # Calcular la posición en la esquina inferior izquierda (dejando un pequeño margen)
    textX = (weight - textsize[0] - 20)  # Se deja espacio para el margen y el texto
    textY = height - 30  # Se deja un margen

    cv.putText(image, text_claro, (textX, textY), font, fontScale, color_bright, thickness, cv.LINE_AA)
    cv.putText(image, text_oscuro, (textX, textY+20), font, fontScale, color_dark, thickness, cv.LINE_AA)



    # Mostrar la imagen con los puntos resaltados
    cv.imshow("Camara", image)

    # Salir del bucle si se presiona la tecla 'Esc' (código ASCII 27)
    if cv.waitKey(20) == 27:
        break

# Liberar recursos
video.release()
cv.destroyAllWindows()

Tarea - Pintar la region 8x8 más clara y la más oscura

In [31]:
def find_brightest_and_darkest_8x8_region(image, find_max=True):
    extreme_region = None
    extreme_brightness = -1 if find_max else 256  # Inicializar con valores apropiados

    for y in range(image.shape[0] - 7):
        for x in range(image.shape[1] - 7):
            region = image[y:y+8, x:x+8]
            region_brightness = np.mean(region)
            if (find_max and region_brightness > extreme_brightness) or \
               (not find_max and region_brightness < extreme_brightness):
                extreme_brightness = region_brightness
                extreme_region = (x, y)

    return extreme_region, extreme_brightness

video = cv.VideoCapture(0)

font = cv.FONT_HERSHEY_SIMPLEX
fontScale = 0.5
color_dark = (255, 0, 0)
color_bright = (0, 255, 0)
thickness = 2

while True:
        ret, image = video.read()
        if not ret:
            print("Error al leer el frame.")
            break

        gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

        darkest_region, _ = find_brightest_and_darkest_8x8_region(gray_image, find_max=False)
        brightest_region, _ = find_brightest_and_darkest_8x8_region(gray_image, find_max=True)

        if darkest_region:
            cv.rectangle(image, darkest_region, (darkest_region[0] + 8, darkest_region[1] + 8), color_dark, thickness)

        if brightest_region:
            cv.rectangle(image, brightest_region, (brightest_region[0] + 8, brightest_region[1] + 8), color_bright, thickness)

        height, width = image.shape[:2]
        text_claro = "Region 8x8 mas clara"
        text_oscuro = "Region 8x8 mas oscura"

        textsize_claro = cv.getTextSize(text_claro, font, fontScale=fontScale, thickness=thickness)[0]
        textsize_oscuro = cv.getTextSize(text_oscuro, font, fontScale=fontScale, thickness=thickness)[0]

        textX = 10
        textY = height - textsize_claro[1] - 10

        cv.putText(image, text_claro, (textX, textY-10-textsize_claro[1]), font, fontScale, color_bright, thickness, cv.LINE_AA)
        cv.putText(image, text_oscuro, (textX, textY), font, fontScale, color_dark, thickness, cv.LINE_AA)

        cv.imshow("cam", image)

        if cv.waitKey(20) == 27:
            break

video.release()
cv.destroyAllWindows()